# Introduction
As someone looking to buy a home in Utah right now, the relative affordability of Utah's largest counties is of direct concern to me, as is the general economic wellbeing of my home state. In order to get a perception of trends in housing affordability, I used county-level economic data from the Federal Reserve Bank of St. Louis, and county-level housing price data from the Utah open data catalog.

While the visualization will be done in R, I chose to do the large amount of data tidying in Python, simply because it is the environment with which I am more comfortable.

# 1 Load Data
## 1.1 Package Importing
For todying this dataset, I will need pandas, numpy, and glob.

In [1]:
#data comes from the Federal Reserve Bank of St. Louis (https://fred.stlouisfed.org/categories/30154) 
#and the Utah Open Data Catalog (https://opendata.utah.gov/Social-Services/Average-Price-3-Bedroom-Homes-By-County-In-Utah-19/5icz-nmjb)

#import necessary packages
import pandas as pd
import numpy as np
import glob

## 1.2 Data Importing
The first dataset we will require is the 10-year change in housing price dataset from the Utah Open Data Catalog. The dataset is complete, but very poorly organized, so it will require quite a bit of work to get it into a usable state. The first step in that process will be to transpose it, so the the columns are counties, and the rows are dates.

In [2]:
#read in housing data
housing = pd.read_csv("10-year_Change_in_Average_3_Bedroom_Home_Prices_in_Utah_Counties.csv")

#transpose house cost table (counties as columns)
housing = housing.transpose()

Next, we will reset the index, and identify the row to use as the columm labels, dopping that row from the dataset. 

In [3]:
#reset the index, name the columns based on the cities, drop irrelevant info
housing = housing.reset_index()
housing.columns = housing.iloc[0]
housing = housing.drop([0, 230])

In order for the data to be usable, we will need the dates to be read as datetime objects, so that we can later merge it with the other dataframes. We will also fix the column names, so that we are beginning with some apropriate consistency in nomenclature.

In [4]:
#convert Month column to datetime
housing["Month"] = pd.to_datetime(housing["Month"], format = "%Y-%m")

In [5]:
#append "_houseprice" to each column name
housing.columns = [str(col) + '_houseprice' for col in housing.columns]
housing.rename(columns={'Month_houseprice':'Month', "Salt Lake_houseprice":"SaltLake_houseprice"}, inplace=True)

In [6]:
#set the "Month" column as the index of the housing df
housing.set_index("Month", inplace = True)

We will next separate all of the data out into separate dataframes, to be individually merged with its respective income dataframe. They will later be concatenated vertically. 

In [7]:
#create separate dfs for each county
Davis_housing = pd.DataFrame(housing["Davis_houseprice"]).rename({"Davis_houseprice":"houseprice"}, axis = 1)
SaltLake_housing = pd.DataFrame(housing["SaltLake_houseprice"]).rename({"SaltLake_houseprice":"houseprice"}, axis = 1)
Tooele_housing = pd.DataFrame(housing["Tooele_houseprice"]).rename({"Tooele_houseprice":"houseprice"}, axis = 1)
Utah_housing = pd.DataFrame(housing["Utah_houseprice"]).rename({"Utah_houseprice":"houseprice"}, axis = 1)
Wasatch_housing = pd.DataFrame(housing["Wasatch_houseprice"]).rename({"Wasatch_houseprice":"houseprice"}, axis = 1)
Washington_housing = pd.DataFrame(housing["Washington_houseprice"]).rename({"Washington_houseprice":"houseprice"}, axis = 1)
Weber_housing = pd.DataFrame(housing["Weber_houseprice"]).rename({"Weber_houseprice":"houseprice"}, axis = 1)

We will use glob to create a list of all of the CSV files containing household income data, which we will read in, and assign to individual dataframes. 

In [8]:
#use glob to get list of all MHI CSVs
MHI_list = glob.glob("*MHI.csv")

In [9]:
#read in MHI data
DavisMHI = pd.read_csv('DavisCountyMHI.csv', names = ["Month", "MHI"], header = 0)
SaltLakeMHI = pd.read_csv('SaltLakeCountyMHI.csv', names = ["Month", "MHI"], header = 0)
TooeleMHI = pd.read_csv('TooeleCountyMHI.csv', names = ["Month", "MHI"], header = 0)
UtahMHI = pd.read_csv('UtahCountyMHI.csv', names = ["Month", "MHI"], header = 0)
WasatchMHI = pd.read_csv('WasatchCountyMHI.csv', names = ["Month", "MHI"], header = 0)
WashingtonMHI = pd.read_csv('WashingtonCountyMHI.csv', names = ["Month", "MHI"], header = 0)
WeberMHI = pd.read_csv('WeberCountyMHI.csv', names = ["Month", "MHI"], header = 0)

# 2 Data Analysis
## 2.1 Data Merging
Next, we will create lists of the housing and income dataframes to allow for easy iteration over the entire set of each. The first such iteration will be to set the months to datetime formats, and fix other formatting issues in the income dataframes.

In [10]:
#create a list of the MHI and housing dfs
MHI_dfs = [DavisMHI, SaltLakeMHI, TooeleMHI, UtahMHI, WasatchMHI, WashingtonMHI, WeberMHI]
housing_dfs = [Davis_housing, SaltLake_housing, Tooele_housing, Utah_housing, Wasatch_housing, Washington_housing, Weber_housing]

In [11]:
#Convert Month col to datetime, set that col as index, and replace "." with NaN
for df in MHI_dfs:
    df["Month"] = pd.to_datetime(df["Month"])
    df.set_index("Month", inplace = True)
    df.replace(".", np.nan, inplace = True)

In [12]:
#convert MHI col to numeric
for df in MHI_dfs:
    df.iloc[:,0] = pd.to_numeric(df.iloc[:,0], errors = 'coerce')

## 2.2 Time Series Resampling
The housing dataframe has a monthly frequency, but the income frames have a yearly sampling frequency. To solve this, I will resample the housing to monthly (month start, or "MS" frequency). To keep the plots smooth, I will apply a linear interpolation on the income data. This is not perfect, but it is the closest I can get to monthly sampling frequency. 

In [13]:
#resample to monthly using month start offset rule ("MS")
DavisMHI = DavisMHI.resample("MS").asfreq()
SaltLakeMHI = SaltLakeMHI.resample("MS").asfreq()
TooeleMHI = TooeleMHI.resample("MS").asfreq()
UtahMHI = UtahMHI.resample("MS").asfreq()
WasatchMHI = WasatchMHI.resample("MS").asfreq()
WashingtonMHI = WashingtonMHI.resample("MS").asfreq()
WeberMHI = WeberMHI.resample("MS").asfreq()

In [14]:
#intepolate county-level MHI data to get monthly MHI estimates. make sure columns have appropriate names
DavisMHI.interpolate(inplace = True)
SaltLakeMHI.interpolate(inplace = True)
TooeleMHI.interpolate(inplace = True)
UtahMHI.interpolate(inplace = True)
WasatchMHI.interpolate(inplace = True)
WashingtonMHI.interpolate(inplace = True)
WeberMHI.interpolate(inplace = True)

I will attach the county name to each dataframe, so that when I combine them all back together (vertically), the county will still be connected to each value. 

In [15]:
#to make the data tidy, create a column "county" for each df, and populate it with the county
DavisMHI["county"] = "Davis"
SaltLakeMHI["county"] = "Salt Lake"
TooeleMHI["county"] = "Tooele"
UtahMHI["county"] = "Utah"
WasatchMHI["county"] = "Wasatch"
WashingtonMHI["county"] = "Washington"
WeberMHI["county"] = "Weber"

Now that each individual dataframe is cleaned up, I will merge each county's income data with its housing price data. Then I will vertically concatenate them into a tall dataframe.  

In [16]:
#outer merge the housing and MHI csv on dt index
Davis_merge = DavisMHI.merge(Davis_housing, how = "inner", left_index = True, right_index = True)
SaltLake_merge = SaltLakeMHI.merge(SaltLake_housing, how = "inner", left_index = True, right_index = True)
Tooele_merge = TooeleMHI.merge(Tooele_housing, how = "inner", left_index = True, right_index = True)
Utah_merge = UtahMHI.merge(Utah_housing, how = "inner", left_index = True, right_index = True)
Wasatch_merge = WasatchMHI.merge(Wasatch_housing, how = "inner", left_index = True, right_index = True)
Washington_merge = WashingtonMHI.merge(Washington_housing, how = "inner", left_index = True, right_index = True)
Weber_merge = WeberMHI.merge(Weber_housing, how = "inner", left_index = True, right_index = True)

In [17]:
#vertically concatenate all merge dfs
merge_dfs = [Davis_merge, SaltLake_merge, Tooele_merge, Utah_merge, Wasatch_merge, Washington_merge, Weber_merge]
MHI_housing_merge = pd.concat(merge_dfs, axis = 0)
MHI_housing_merge = MHI_housing_merge[["county", "houseprice", "MHI"]]

## 2.3 Calculated Fields
Next, I will calculate the final column that will be visualized, the housing price to income ratio.

In [18]:
MHI_housing_merge['housing_MHI_ratio'] = MHI_housing_merge['houseprice'] / MHI_housing_merge['MHI']

In [19]:
#export full merged df to csv for further analysis in R
MHI_housing_merge.to_csv("housing_MHI_merge.csv")

In [20]:
MHI_housing_merge

,county,houseprice,MHI,housing_MHI_ratio
Month,,,,
1996-04-01,Davis,125000,47629.125000,2.62444
1996-05-01,Davis,123800,47747.000000,2.59283
1996-06-01,Davis,122900,47864.875000,2.56764
1996-07-01,Davis,122900,47982.750000,2.56134
1996-08-01,Davis,123100,48100.625000,2.55922
...,...,...,...,...
2014-12-01,Weber,152400,58828.250000,2.59059
2015-01-01,Weber,152700,58786.000000,2.59756
2015-02-01,Weber,153200,59117.833333,2.59143


## 2.4 Dataframe Melt & Export
Lastly, I will melt the dataframe so that the column headers, which contain the measurements, will be represented as a single column, and so that each row contains a single measurement for a single day in a single county. This dataframe will be written to a CSV for further analysis and visualization in R. 

In [21]:
#melt the df so that column headers are stored in the column "metric"
MHI_housing_melt = pd.melt(MHI_housing_merge.reset_index(), value_vars = ["houseprice", "MHI", "housing_MHI_ratio"], id_vars = ["Month", "county"]).set_index("Month")
MHI_housing_melt

,county,variable,value
Month,,,
1996-04-01,Davis,houseprice,125000
1996-05-01,Davis,houseprice,123800
1996-06-01,Davis,houseprice,122900
1996-07-01,Davis,houseprice,122900
1996-08-01,Davis,houseprice,123100
...,...,...,...
2014-12-01,Weber,housing_MHI_ratio,2.59059
2015-01-01,Weber,housing_MHI_ratio,2.59756
2015-02-01,Weber,housing_MHI_ratio,2.59143


In [22]:
MHI_housing_melt.to_csv("housing_MHI_melt.csv")